#0) Imports and Installs

In [1]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

import pandas as pd
import requests


# Question 1: IPO Filings Web Scraping and Data Processing

In [2]:
url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url)#, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [3]:
ipoFilings = ipo_dfs[0]
ipoFilings['Filing Date'] = pd.to_datetime(ipoFilings['Filing Date'])
ipoFilings.set_index('Filing Date', inplace=True)
ipoFilings = ipoFilings[ipoFilings.index.year==2023]
ipoFilings

,Symbol,Company Name,Price Range,Shares Offered
Filing Date,,,,
2023-12-29,LEC,Lafayette Energy Corp,$3.50 - $4.50,1200000
2023-12-29,EPSM,Epsium Enterprise Limited,-,-
2023-12-28,ONDR,"Sushi Ginza Onodera, Inc.",$7.00 - $8.00,1066667
2023-12-27,JDZG,Jiade Limited,$4.00 - $5.00,2200000
2023-12-22,CHLW,Chun Hui Le Wan International Holding Group Ltd,-,-
...,...,...,...,...
2023-01-31,FBGL,FBS Global Limited,$4.00 - $5.00,1875000
2023-01-24,THNK,"T1V, Inc.",$4.00 - $6.00,3300000
2023-01-23,RPET,New Ruipeng Pet Group Inc.,-,-


In [4]:
def extractPrice(x):
    if len(x.split('-')) > 1:
        return np.mean([float(i.strip()[1:]) for i in x.split('-') if len(i)>1])
    elif x=='-':
        return np.nan
    else:
        return float(x[1:])
    
ipoFilings['Price Range'] = ipoFilings['Price Range'].apply(lambda x: extractPrice(x))
ipoFilings['Shares Offered'] = ipoFilings['Shares Offered'].apply(lambda x: float(x) if x!='-' else np.nan)
ipoFilings['Weekday'] = ipoFilings.index.weekday
ipoFilings = ipoFilings[ipoFilings['Weekday']==4]
ipoFilings['Shares_offered_value'] = ipoFilings['Price Range'] * ipoFilings['Shares Offered']
ipoFilings.Shares_offered_value.sum()

/Users/whysocurious/miniforge3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3408863864.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipoFilings['Price Range'] = ipoFilings['Price Range'].apply(lambda x: extractPrice(x))
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3408863864.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

285700000.0

In [5]:
print (ipoFilings.shape)
ipoFilings.isna().sum()

(32, 6)


Symbol                  0
Company Name            0
Price Range             7
Shares Offered          7
Weekday                 0
Shares_offered_value    7
dtype: int64

# Question 2: IPOs "Fixed days hold" strategy

In [6]:
url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url)#, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [7]:
ipos_2023 = ipo_dfs[0]
ipos_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [8]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url)#, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [9]:
ipos_2024 = ipo_dfs[0]
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      64 non-null     object
 1   Symbol        64 non-null     object
 2   Company Name  64 non-null     object
 3   IPO Price     64 non-null     object
 4   Current       64 non-null     object
 5   Return        64 non-null     object
dtypes: object(6)
memory usage: 3.1+ KB


In [10]:
print (ipos_2023.shape, ipos_2024.shape)

(154, 6) (64, 6)


In [11]:
stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)
stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'])
stacked_ipos_df = stacked_ipos_df[stacked_ipos_df['IPO Date'] < pd.to_datetime('2024-03-01')]
stacked_ipos_df = stacked_ipos_df[stacked_ipos_df.Symbol != 'RYZB']
stacked_ipos_df

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
33,2024-02-27,SMXT,"SolarMax Technology, Inc.",$4.00,$10.42,160.50%
34,2024-02-22,VHAI,Vocodia Holdings Corp,$4.25,$0.16,-96.29%
35,2024-02-21,DYCQ,DT Cloud Acquisition Corporation,$10.00,$10.35,3.50%
36,2024-02-16,CHRO,Chromocell Therapeutics Corp,$6.00,$1.77,-70.50%
37,2024-02-14,UMAC,"Unusual Machines, Inc.",$4.00,$1.30,-67.50%
...,...,...,...,...,...,...
213,2023-01-25,QSG,QuantaSing Group Ltd,$12.50,$3.15,-74.80%
214,2023-01-20,CVKD,"Cadrenal Therapeutics, Inc.",$5.00,$0.47,-90.19%
215,2023-01-13,SKWD,"Skyward Specialty Insurance Group, Inc.",$15.00,$37.58,150.53%
216,2023-01-13,ISRL,Israel Acquisitions Corp,$10.00,$10.91,9.10%


In [12]:
ipoOHCLV = pd.DataFrame()
for symb in stacked_ipos_df.Symbol.unique():
    if symb == 'IBAC':
        symb='IBACU'
    else: 
        pass
    dftmp = yf.download(tickers = symb,
                         period = "max",
                         interval = "1d").loc[:,['Adj Close']]
    
    dftmp.rename(columns={'Adj Close':symb}, inplace=True)
    dftmp = dftmp[dftmp.index >= stacked_ipos_df.loc[stacked_ipos_df.Symbol==symb,'IPO Date'].values[0]]
    dftmp = dftmp[dftmp.index < pd.to_datetime('2024-03-01')]
    
    ipoOHCLV = pd.concat([ipoOHCLV, dftmp],axis=1)

ipoOHCLV

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,SMXT,VHAI,DYCQ,CHRO,UMAC,TBBB,MGX,HLXB,TELO,KYTX,...,ASST,CETU,TXO,BREA,GNLX,QSG,CVKD,SKWD,ISRL,MGOL
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.74,NaN,20.627850,2.82,5.980,12.40,1.99,20.059999,NaN,1.84
2023-02-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.83,NaN,20.699820,3.15,6.090,12.70,1.98,20.790001,NaN,1.91
2023-02-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.77,NaN,20.681828,3.36,5.990,13.15,2.12,20.650000,NaN,1.85
2023-02-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.79,NaN,20.897732,2.88,6.445,12.71,2.16,20.639999,NaN,1.99
2023-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.66,NaN,20.627850,2.60,6.200,12.80,2.20,20.080000,NaN,2.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.20,18.490000,NaN,2.85
2023-01-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.100000,NaN,4.65
2023-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.480000,NaN,4.90


In [19]:
ipoOHCLV.sort_index(inplace=True)
dfIPO_MaxGrwth_75ile = pd.DataFrame()
for symb in ipoOHCLV.columns:
    dftmp = ipoOHCLV[[symb]].dropna()
    for i in range(1,31):
        dftmp[symb+'_grw_'+str(i)+'d'] = (dftmp[symb].shift(-i) / dftmp[symb]) - 1
    dfIPO_MaxGrwth_75ile = pd.concat([dfIPO_MaxGrwth_75ile,
                                      pd.DataFrame([symb,
                                                    float(dftmp.describe().drop(symb,axis=1).T[['mean']].mean().values[0]),
                                                    float(dftmp.describe().drop(symb,axis=1).T[['50%']].mean().values[0]),
                                                    float(dftmp.describe().drop(symb,axis=1).T[['75%']].mean().values[0]),
                                                    dftmp.describe().drop(symb,axis=1).T[['mean']].idxmax().values[0],
                                                    dftmp.describe().drop(symb,axis=1).T[['50%']].idxmax().values[0],
                                                    dftmp.describe().drop(symb,axis=1).T[['75%']].idxmax().values[0]]).T])
    
dfIPO_MaxGrwth_75ile.columns = ['Symbol','meanVal','50ileVal','75ileVal','mean','50ile','75ile']
dfIPO_MaxGrwth_75ile.dropna(inplace=True)
dfIPO_MaxGrwth_75ile['DaysMean'] = dfIPO_MaxGrwth_75ile['mean'].apply(lambda x: int(x.split('_')[-1][:-1]))
dfIPO_MaxGrwth_75ile['Days50'] = dfIPO_MaxGrwth_75ile['50ile'].apply(lambda x: int(x.split('_')[-1][:-1]))
dfIPO_MaxGrwth_75ile['Days75'] = dfIPO_MaxGrwth_75ile['75ile'].apply(lambda x: int(x.split('_')[-1][:-1]))

dfIPO_MaxGrwth_75ile['meanVal'] = dfIPO_MaxGrwth_75ile['meanVal'].astype(float)
dfIPO_MaxGrwth_75ile['50ileVal'] = dfIPO_MaxGrwth_75ile['50ileVal'].astype(float)
dfIPO_MaxGrwth_75ile['75ileVal'] = dfIPO_MaxGrwth_75ile['75ileVal'].astype(float)

dfIPO_MaxGrwth_75ile.describe()

,DaysMean,Days50,Days75
count,180.000000,180.000000,180.000000
mean,15.500000,13.155556,19.866667
std,12.911603,12.619849,10.006925
min,1.000000,1.000000,1.000000
25%,1.000000,1.000000,11.000000
50%,14.500000,7.500000,23.000000
75%,30.000000,28.250000,30.000000
max,30.000000,30.000000,30.000000


In [20]:
dfIPO_MaxGrwth_75ile.describe()

,meanVal,50ileVal,75ileVal,DaysMean,Days50,Days75
count,180.000000,180.000000,180.000000,180.000000,180.000000,180.000000
mean,-0.006926,-0.038368,0.075444,15.500000,13.155556,19.866667
std,0.112152,0.103855,0.131782,12.911603,12.619849,10.006925
min,-0.463858,-0.495648,-0.340358,1.000000,1.000000,1.000000
25%,-0.056850,-0.091169,0.004939,1.000000,1.000000,11.000000
50%,0.002379,-0.013231,0.048883,14.500000,7.500000,23.000000
75%,0.033331,0.003636,0.117741,30.000000,28.250000,30.000000
max,0.593143,0.321089,0.660706,30.000000,30.000000,30.000000


## Question 3: Is Growth Concentrated in the Largest Stocks?

In [21]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

In [22]:
NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [23]:
import time

stocks_largest = pd.DataFrame({'A' : []})

for i,ticker in enumerate(LARGEST_STOCKS):
    print(i,ticker)

    # Work with stock prices
    historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")
    historyPrices = historyPrices[(historyPrices.index > pd.to_datetime('2013-12-25'))&
                                  (historyPrices.index < pd.to_datetime('2024-1-1'))]
    
    # generate features for historical prices, and what we want to predict
    historyPrices['Ticker'] = ticker
    historyPrices['Date'] = historyPrices.index.date

    i = 7
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)

    if stocks_largest.empty:
        stocks_largest = historyPrices
    else:
        stocks_largest = pd.concat([stocks_largest, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/4021536491.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Ticker'] = ticker
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/4021536491.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Date'] = historyPrices.index.date
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/4021536491.py:20: SettingWithCopyWarni

1 AAPL


[*********************100%%**********************]  1 of 1 completed
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/4021536491.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Ticker'] = ticker
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/4021536491.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Date'] = historyPrices.index.date
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/4021536491.py:20: SettingWithCopyWarni

2 GOOG


...

[*********************100%%**********************]  1 of 1 completed
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/4021536491.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Ticker'] = ticker
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/4021536491.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Date'] = historyPrices.index.date
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/4021536491.py:20: SettingWithCopyWarni

31 HINDUNILVR.NS


[*********************100%%**********************]  1 of 1 completed
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/4021536491.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Ticker'] = ticker
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/4021536491.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Date'] = historyPrices.index.date
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/4021536491.py:20: SettingWithCopyWarni

32 LT.NS


[*********************100%%**********************]  1 of 1 completed
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/4021536491.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Ticker'] = ticker
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/4021536491.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Date'] = historyPrices.index.date
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/4021536491.py:20: SettingWithCopyWarni

In [24]:
import time

stocks_large = pd.DataFrame({'A' : []})

for i,ticker in enumerate(LARGE_STOCKS):
    print(i,ticker)

    # Work with stock prices
    historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")
    historyPrices = historyPrices[(historyPrices.index > pd.to_datetime('2013-12-25'))&
                                  (historyPrices.index < pd.to_datetime('2024-1-1'))]
    
    # generate features for historical prices, and what we want to predict
    historyPrices['Ticker'] = ticker
    historyPrices['Date'] = historyPrices.index.date

    i = 7
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)

    if stocks_large.empty:
        stocks_large = historyPrices
    else:
        stocks_large = pd.concat([stocks_large, historyPrices], ignore_index=True)

0 PRX.AS


[*********************100%%**********************]  1 of 1 completed
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3069493816.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Ticker'] = ticker
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3069493816.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Date'] = historyPrices.index.date
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3069493816.py:20: SettingWithCopyWarni

1 CDI.PA


[*********************100%%**********************]  1 of 1 completed
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3069493816.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Ticker'] = ticker
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3069493816.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Date'] = historyPrices.index.date
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3069493816.py:20: SettingWithCopyWarni

2 AIR.PA


...

[*********************100%%**********************]  1 of 1 completed
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3069493816.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Ticker'] = ticker
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3069493816.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Date'] = historyPrices.index.date
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3069493816.py:20: SettingWithCopyWarni

32 KOTAKBANK.NS


[*********************100%%**********************]  1 of 1 completed
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3069493816.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Ticker'] = ticker
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3069493816.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Date'] = historyPrices.index.date
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3069493816.py:20: SettingWithCopyWarni

33 TITAN.NS


[*********************100%%**********************]  1 of 1 completed
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3069493816.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Ticker'] = ticker
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3069493816.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historyPrices['Date'] = historyPrices.index.date
/var/folders/8x/4tbqnfzd0tz4gn4chf6rr2ch0000gp/T/ipykernel_17273/3069493816.py:20: SettingWithCopyWarni

In [31]:
df7dGrowthCompare = pd.merge(stocks_largest.pivot_table(index='Date',
                                             values='growth_7d',
                                             aggfunc=np.nanmean),
                             stocks_large.pivot_table(index='Date',
                                             values='growth_7d',
                                             aggfunc=np.nanmean),
                             how = 'left', 
                             left_index=True,
                             right_index=True, 
                             suffixes= ['_largest','_large'])
df7dGrowthCompare['compare'] = df7dGrowthCompare.growth_7d_largest > df7dGrowthCompare.growth_7d_large
np.nansum(df7dGrowthCompare['compare']) / 2595

0.5252408477842004

## Question 4: Trying Another Technical Indicators strategy

In [32]:
dfQ4 = pd.read_parquet('stocks_df_combined_trunc_2014_2023.parquet.brotli')
print (dfQ4.shape)
dfQ4.head()

(80762, 202)


,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
7011,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014,2014-01-01,3,...,0.964302,0.992998,0.970030,1.158676,NaN,NaN,NaN,NaN,NaN,NaN
7012,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014,2014-01-01,4,...,0.958139,0.984707,0.961500,1.143209,NaN,NaN,NaN,NaN,NaN,NaN
7013,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014,2014-01-01,0,...,0.953798,0.998223,0.968951,1.168236,NaN,NaN,NaN,NaN,NaN,NaN
7014,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014,2014-01-01,1,...,0.958653,0.993430,0.977598,1.097648,NaN,NaN,NaN,NaN,NaN,NaN
7015,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014,2014-01-01,2,...,0.955161,0.973383,0.974977,1.100781,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
dfCCI = dfQ4.loc[(dfQ4.cci > 200)&(dfQ4.Weekday==4),['Date','Ticker','growth_future_5d']]#.pivot_table(index='Date', columns='Ticker',values='growth_future_5d')
dfCCI.set_index('Date', inplace=True)
dfCCI['profit'] = 1000 * (dfCCI.growth_future_5d - 1)
print (dfCCI['profit'].sum())
dfCCI[dfCCI.Ticker=='MSFT']

1048.2628918177302


,Ticker,growth_future_5d,profit
Date,,,
2014-05-30,MSFT,1.013190,13.190094
2014-09-05,MSFT,1.017207,17.207263
2015-04-24,MSFT,1.016503,16.503151
2015-10-02,MSFT,1.033794,33.793947
2015-10-23,MSFT,0.995650,-4.350092
2016-01-29,MSFT,0.910510,-89.490064
2016-10-21,MSFT,1.003520,3.519788
2017-01-27,MSFT,0.968075,-31.924599
2017-04-21,MSFT,1.031024,31.024462


In [56]:
print (dfCCI['profit'].sum())


1048.2628918177302
